In [3]:
%%bash
# cp -r /home/michal/ParlaMint/{SRB,BiH} ..
# cp /home/rupnik/parlamint/task16/utils.py /home/rupnik/parlamint/task17

# Bosnian session splitting:

In [4]:
lang = "BiH"

from pathlib import Path
import shutil
import os

location_of_original_data = f"/home/rupnik/parlamint/{lang}/"

from utils import parse_meta_file, parse_text_file
os.system(f"mkdir {location_of_original_data}S_data")
metafiles = Path(location_of_original_data).glob("*_meta.tsv")

from utils import correct_id


for m in metafiles:
    metadf = parse_meta_file(str(m.absolute()))
    textdf = parse_text_file(str(m.absolute()).replace("meta", "text").replace(".tsv", ".txt"))
    term = metadf.Term.unique()
    assert len(term) == 1, "More than one term in file!"
    term = int(term[0])
    for session in metadf.Session.unique():
        subset = metadf[metadf.Session==session].reset_index(drop=True)
        textsubset = textdf[textdf.ID.isin(subset.ID)].reset_index(drop=True)
        
        filename = f"meta_T{term:02}_S{session}.tsv"
            
        
        # Skip if we have no texts for session:
        if textsubset.shape[0] == 0:
            continue
        
        # # Correcting IDs:
        # subset["ID"] = subset.ID.apply(correct_id)
        # textsubset["ID"] = textsubset.ID.apply(correct_id)

        # Writing data
        subset.to_csv(f"{location_of_original_data}S_data/{filename}",sep="\t", index=False)
        with open(f"{location_of_original_data}S_data/"+filename.replace("meta", "text").replace("tsv", "txt"), "w") as f:
            f.writelines([
                f"{row['ID']} {row['Text']}\n" for i, row in textsubset.iterrows()
            ])

mkdir: cannot create directory ‘/home/rupnik/parlamint/BiH/S_data’: File exists
